In [2]:
struct Delta
    n::Int
    q::Int
    avg::Vector{Vector{Float64}}
end

In [3]:
function Delta(n::Int, q::Int) # Initializes correct size delta
    Delta(n,q,[zeros(Float64, q*q) for _ in 1:n*n])
end

delta

In [4]:
function cart2lin(t::Tuple{Int,Int}, dims::Int) # Cartesian to linear indices conversion
    x,y = t[2], t[1]
    @assert 1 ≤ x ≤ dims && 1 ≤ y ≤ dims
    return y + dims*(x-1)
end

cart2lin (generic function with 1 method)

In [ ]:
function Delta_update!(δ::Delta,i::Int,j::Int,a::Int,b::Int) # Updates the delta structure when the condition is verified
    n = δ.n
    q = δ.q
    δ.avg[cart2lin((i,j),n)][cart2lin((a,b),q)] += 1
    nothing
end

In [ ]:
function compute_stat(data::Matrix{Int64}, q::Int)
	count = size(data, 1)
	n = size(data, 2)
	delta_ijab = Delta(n,q)

	for i in 1:n, j in 1:n, a in 1:q, b in 1:q
		[(data[s, i] == a) && (data[s, j] == b) ? Delta_update!(delta_ijab,i,j,a,b) : nothing for s in 1:count]
	end
	delta_ijab.avg /= count

	return delta_ijab
end;

In [ ]:
function maxabs_vecvec(δ::Delta)
	m = δ.avg
	maxx = -1
	for r in m
		if maximum(abs.(m)) > maxx
			maxx = maximum(abs.(m))
		end
	end
	return maxx
end;